In [1]:
import pandas as pd
from datetime import date, timedelta

In [2]:
file_path = "https://www.dhhs.vic.gov.au/ncov-covid-cases-by-age-group-csv"
df = pd.read_csv(file_path)

In [3]:
df.tail()

,diagnosis_date,agegroup
519078,2022-01-21,10-19_
519079,2022-01-21,70-79
519080,2022-01-21,30-39
519081,2022-01-21,20-29
519082,2022-01-21,0-9


In [4]:
df_pivot = pd.pivot_table(df, index = 'diagnosis_date', columns = 'agegroup', 
                         values = 'agegroup', aggfunc = len, fill_value = 0)

In [5]:
df_pivot.tail()

agegroup,0-9,10-19_,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+
diagnosis_date,,,,,,,,,,
2022-01-17,1046,1044,1635,1894,1352,1082,765,374,173,33
2022-01-18,1252,1168,2024,2237,1627,1213,870,429,217,59
2022-01-19,1385,1232,2042,2414,1646,1245,940,507,176,59
2022-01-20,1178,1063,1825,2105,1455,1160,795,381,198,48
2022-01-21,1176,893,1320,1704,1202,856,691,356,197,37


In [6]:
# check that the latest date's sum matches the number of cases reported by Vic govt
# the latest row's date is always yesterday

yesterday = date.today() - timedelta(days = 1)

# the dates in the pivot table are strings
# so use isoformat() to convert date into string before looking it up in the table
df_pivot.loc[yesterday.isoformat()].sum()

8432

In [7]:
# take only the 0-9 and 10-19 cases, from 5 Dec onwards
# for each date, store the previous date's cases; this matches what I've done for NSW
# eg. on 8 Dec, the Vic cases are those reported on the 7th

first = date(2021, 12, 5)
today = date.today()

df_0_19 = pd.DataFrame(columns = ['date', 'cases_0_9', 'cases_10_19'])

curr = first
prev = first - timedelta(days = 1)

while (curr < today):
    
        df_0_19 = df_0_19.append({'date': curr.isoformat(), 'cases_0_9': df_pivot.loc[prev.isoformat(), '0-9'], 
                                  'cases_10_19': df_pivot.loc[prev.isoformat(), '10-19_']}, ignore_index = True)
        prev = curr
        curr = curr + timedelta(days = 1)

In [8]:
df_0_19.tail()

,date,cases_0_9,cases_10_19
43,2022-01-17,1431,1393
44,2022-01-18,1046,1044
45,2022-01-19,1252,1168
46,2022-01-20,1385,1232
47,2022-01-21,1178,1063


In [9]:
df_0_19.to_csv("../data/cases_0_19.csv")